In [1]:
import pandas as pd
from datetime import datetime, timedelta
import time
from time import gmtime, strftime
import ast
import os

# YouTube API
from googleapiclient.discovery import build

In [2]:
os. chdir('C:/Users/Esteban Guerrero/SocialNetworkProject/Iteration2/PythonScripts')
%run ./DataProcessing_PrepareOutputIntoDF.ipynb

In [3]:
os. chdir('C:/Users/Esteban Guerrero/SocialNetworkProject/Iteration2/PythonScripts')
%run ./DataProcessing_TextPreparation.ipynb

In [4]:
# Obtain latest list of videos per channel if video has comments and published after start period

In [5]:
df1 = pd.read_csv('C:/Users/Esteban Guerrero/SocialNetworkProject/Iteration2/Datasets/NetworkFeedRawOutput.csv', index_col = False, encoding = 'ISO-8859-1')
df1 = df1[(df1.ExtrType == 'YouTubeVideoInfo') | (df1.ExtrType == 'VideoInfo-YouTube')].reset_index(drop = True)

In [6]:
df1 = df1.sort_values(by = ['Key1', 'Key2', 'ExtractionDateTime'], ascending = True).reset_index(drop = True)

In [7]:
df1 = df1.sort_values(by = ['Key1', 'Key2', 'ExtractionDateTime'], ascending = True).reset_index(drop = True)
df1 = pd.concat([df1, df1.duplicated(keep = 'last', subset = ['Key1', 'Key2'])], axis = 1)
df1.rename(columns ={0: 'DuplicateFlag'}, inplace = True)

In [8]:
df1 = df1[df1.DuplicateFlag == False].reset_index(drop = True)
df1 = df1.drop(['DuplicateFlag'], axis = 1)

In [9]:
df1['CommentCount'] = 0
df1['publishedAt'] = ''

In [10]:
for i in range(0, len(df1.Output1)):
#for i in range(0, 50):
    df1.iat[i, 6] = ast.literal_eval(df1.iat[i, 2])['VideoStats'].get('commentCount')
    df1.iat[i, 7] = ast.literal_eval(df1.iat[i, 2])['VideoDetails'].get('publishedAt')[0:10]

In [11]:
df1 = df1[(df1.CommentCount > 0) & (df1.CommentCount.notnull()) & (df1.CommentCount.notna())]

In [12]:
df1['publishedAt'] =  pd.to_datetime(df1['publishedAt'], format='%Y-%m-%d')

In [13]:
ndays = 60
start_date = (datetime.now()  - timedelta(ndays-1)).strftime("%Y-%m-%d")
print(start_date)
df1 = df1[df1['publishedAt'] >= start_date]

2021-06-04


In [14]:
df1.head()

,Key1,Key2,Output1,ExtractionDateTime,ExtrMethod,ExtrType,CommentCount,publishedAt
17,UC_vhdXDVvZQGMqQFDgn0w1g,-WPsW2uBtEg,"{'User': 'Espert', 'VideoDetails': {'published...",2021-08-02 16:22:30 -0300,Automatic,VideoInfo-YouTube,14.0,2021-06-10
32,UC_vhdXDVvZQGMqQFDgn0w1g,00QjyVewlX0,"{'User': 'Espert', 'VideoDetails': {'published...",2021-08-02 16:22:30 -0300,Automatic,VideoInfo-YouTube,24.0,2021-06-28
114,UC_vhdXDVvZQGMqQFDgn0w1g,3VgcwXK19Sg,"{'User': 'Espert', 'VideoDetails': {'published...",2021-08-02 16:22:30 -0300,Automatic,VideoInfo-YouTube,74.0,2021-07-26
118,UC_vhdXDVvZQGMqQFDgn0w1g,3afCHxQ2fhs,"{'User': 'Espert', 'VideoDetails': {'published...",2021-08-02 16:22:30 -0300,Automatic,VideoInfo-YouTube,13.0,2021-07-26
123,UC_vhdXDVvZQGMqQFDgn0w1g,3pdNGdVMR-k,"{'User': 'Espert', 'VideoDetails': {'published...",2021-08-02 16:22:30 -0300,Automatic,VideoInfo-YouTube,425.0,2021-06-28


In [15]:
# Initialize API
youTubeApiKey = 'youTubeApiKey'
youtube = build('youtube','v3', developerKey = youTubeApiKey)

In [16]:
# Function to extract from API, video comments data per videoID and return comments dataframe

def CommentsDataPerVideoID(vidID0, ChannelID01):

    # 1 - Extract comments and replies from API per videoID
    vidcomments0 = []
    next_page_token = None
    while 1:
        res = youtube.commentThreads().list(part = 'snippet,replies', videoId = vidID0, 
                                            maxResults = 50, pageToken = next_page_token).execute()
        vidcomments0+= res['items']
        next_page_token = res.get('nextPageToken')

        if next_page_token is None:
            break
    
    # 2 - Create video comments dataframe and extract replies
    vidCommentsDf0 = pd.DataFrame()
    for m in range(0, len(vidcomments0)):
        commentoutput = {'Output1': {'commentContent': {'id': vidcomments0[m]['snippet']['topLevelComment']['id'],
                         'authorDisplayName': TextCleansing(vidcomments0[m]['snippet']['topLevelComment']['snippet']['authorDisplayName']),
                         'textOriginal': TextCleansing(vidcomments0[m]['snippet']['topLevelComment']['snippet']['textOriginal']),
                         'likeCount': vidcomments0[m]['snippet']['topLevelComment']['snippet']['likeCount'],
                         'publishedAt': vidcomments0[m]['snippet']['topLevelComment']['snippet']['publishedAt'],
                         'updatedAt': vidcomments0[m]['snippet']['topLevelComment']['snippet']['updatedAt']}}}

        rply = vidcomments0[m].get('replies')
        if rply != None:
            rplL = rply['comments']
            for rp in rplL:
                rplyDict = {'Output1' : {'reply':  {'etag': rp.get('etag'), 'id': rp.get('id'),
                                                    'videoId': rp.get('snippet').get('videoId'),
                                                    'textOriginal': TextCleansing(rp.get('snippet').get('textOriginal')),
                                                    'parentId': rp.get('snippet').get('parentId'),
                                                    'authorDisplayName': TextCleansing(rp.get('snippet').get('authorDisplayName')),
                                                    'likeCount': rp.get('snippet').get('likeCount'),
                                                    'publishedAt': rp.get('snippet').get('publishedAt')}
                                        }}
                vidCommentsDf0 = vidCommentsDf0.append(rplyDict, ignore_index=True, sort = False)
        
        vidCommentsDf0 = vidCommentsDf0.append(commentoutput, ignore_index=True, sort = False)
    
    # 3 - Finalize comments dataframe 
    vidCommentsDf0['Key1'] = ChannelID01
    vidCommentsDf0['Key2'] = vidID0  
    
    # 4 - Return comments dataframe per video
    
    return vidCommentsDf0

In [17]:
chList = df1.Key1.drop_duplicates().values.tolist()
videoCommentsDF = pd.DataFrame()

for ChannelID in chList:
    print('***', str(ChannelID), '\nstart\t', datetime.now())
    videoIdList = df1['Key2'][df1.Key1 == ChannelID].drop_duplicates().values.tolist()
    
    x = 1    
    vidcommDF = pd.DataFrame()    
    for vidID in videoIdList:
        
        vidcommDF = CommentsDataPerVideoID(vidID, ChannelID)
        videoCommentsDF = videoCommentsDF.append(vidcommDF, ignore_index = True)
        
        if (x % 100 == 0):
            time.sleep(3)
        
        x = x + 1
    
    YToutputCommDF = PrepareOutputDF('K12', videoCommentsDF, 'Automatic', 'VideoComments-YouTube')
    
    # Write and store channel video list, video details and comments
    DF = pd.read_csv('C:/Users/Esteban Guerrero/SocialNetworkProject/Iteration2/Datasets/NetworkFeedRawOutput.csv', index_col = False, encoding = 'ISO-8859-1')
    DF = DF.append([YToutputCommDF], ignore_index=True)
    DF.to_csv('C:/Users/Esteban Guerrero/SocialNetworkProject/Iteration2/Datasets/NetworkFeedRawOutput.csv', index = False)
    
    print('end\t', datetime.now()) 

*** UC_vhdXDVvZQGMqQFDgn0w1g 
start	 2021-08-02 16:23:10.000945
end	 2021-08-02 16:25:17.223360
*** UCwKZnJ48oXRCCGCQbL1BPSA 
start	 2021-08-02 16:25:17.223360
end	 2021-08-02 16:25:49.036331
